I use this file to write new ENVI data.

In [1]:
from misc.sen2 import writeENVI
from raster import Raster
import numpy as np

This is where the data will be written.

In [2]:
output_file = 'test_C11659/0920.bin'

These are files where new data are calculated

In [3]:
pre_fire_f = 'fire_C11659/S2C_MSIL2A_20250609T192931_N0511_R142_T09UYU_20250610T002612_cloudfree.bin_MRAP_C11659.bin'

post_fire_f = 'fire_C11659/S2A_MSIL2A_20250919T193831_N0511_R142_T09UYU_20250920T013316_cloudfree.bin_MRAP_C11659.bin'

Perform Calculations

In [4]:
#1. pre and post fire objects

pre_fire_obj = Raster(pre_fire_f)
pre_fire_dat = pre_fire_obj.read_bands('all')

post_fire_obj = Raster(post_fire_f)
post_fire_dat = post_fire_obj.read_bands('all')

In [5]:
#2. difference product

from change_detection import change_detection

change_dat = change_detection(
    pre_X=pre_fire_dat,
    post_X=post_fire_dat
)

In [6]:
#3. dNBR SWIR B12

from barc import dNBR

_, _, dnbr12 = dNBR(
    NIR_pre=pre_fire_obj.get_band(8),
    SWIR_pre=pre_fire_obj.get_band(12),

    NIR_post=post_fire_obj.get_band(8),
    SWIR_post=post_fire_obj.get_band(12)
)

In [7]:
#4. dNBR SWIR 11

from barc import dNBR

_, _, dnbr11 = dNBR(
    NIR_pre=pre_fire_obj.get_band(8),
    SWIR_pre=pre_fire_obj.get_band(11),

    NIR_post=post_fire_obj.get_band(8),
    SWIR_post=post_fire_obj.get_band(11)
)

Merge data and band info

In [8]:
new_dat = np.dstack([
    pre_fire_dat,
    post_fire_dat,
    change_dat,
    dnbr12,
    dnbr11
])

In [9]:
new_band_names = [
    'B12_pre',
    'B11_pre',
    'B9_pre',
    'B8_pre',

    'B12_post',
    'B11_post',
    'B9_post',
    'B8_post',

    'B12_diff',
    'B11_diff',
    'B9_diff',
    'B8_diff',

    'dnbr12',

    'dnbr11'
]

In [10]:
new_dat.shape

(2992, 2708, 14)

Write new data to file

In [11]:
writeENVI(
    output_filename = output_file,
    data = new_dat,
    mode ='new',
    ref_filename = pre_fire_f,
    band_names = new_band_names
)